# MAST30034 Project 1
## Preprocessing taxis

In [2]:
from functools import reduce 
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/15 20:59:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Downloading taxi data for yellow and green taxis from jan 2018 - dec 2019

In [23]:
from urllib.request import urlretrieve
import os

# from the current `tute_1` directory, go back two levels to the `MAST30034` directory
output_relative_dir = '../data/raw'

# check if it exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)

TAXI  = ['yellow', 'green']
YEARS = ['2018', '2019']
MONTHS = range(1, 13)
for taxi in TAXI:
    # this is the URL template as of 07/2022
    URL_TEMPLATE = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi}_tripdata_"#year-month.parquet

    print(f"Begin {taxi} taxi data")
    for year in YEARS:
        for month in MONTHS:
            # 0-fill i.e 1 -> 01, 2 -> 02, etc
            month = str(month).zfill(2) 
            print(f"Begin month {month}/{year}")

            # generate url
            url = f'{URL_TEMPLATE}{year}-{month}.parquet'
            # generate output location and filename
            output_dir = f"{output_relative_dir}/{taxi}_tripdata_{year}-{month}.parquet"
            # download
            urlretrieve(url, output_dir) 

            print(f"Completed month {month}/{year}")

Begin yellow taxi data
Begin month 01/2018
Completed month 01/2018
Begin month 02/2018


KeyboardInterrupt: 

The Green taxi service has an extra column called ehail_fee, to help combine the dataframes, all yellow taxi dataframes will have a ehail_fee column appended to it with 0 for each row as yellow taxis despite having an ehail service, dont charge for it.

Green taxi services also has trip_type whereas yellow taxis do not, given that yellow taxis can both hail from the street as well as being pre organised, it is impossible to populate this feild with any real meaningful data thus it will be removd from the green taxi dataframes.

Yellow taxis also have an airport_fee column, dispite Green taxis being able to go the airport, they do log any infomation of airport_fees, similar to the trip_type situation, it is unreasonable to populate this column with meaningful data and thus wil be removed from all yellow taxi dataframes

In [3]:
from pyspark.sql.functions import lit

Gjan19 = spark.read.parquet('../data/raw/green_tripdata_2019-01.parquet').drop("trip_type")
Gfeb19 = spark.read.parquet('../data/raw/green_tripdata_2019-02.parquet').drop("trip_type")
Gmar19 = spark.read.parquet('../data/raw/green_tripdata_2019-03.parquet').drop("trip_type")
Gapr19 = spark.read.parquet('../data/raw/green_tripdata_2019-04.parquet').drop("trip_type")
Gmay19 = spark.read.parquet('../data/raw/green_tripdata_2019-05.parquet').drop("trip_type")
Gjun19 = spark.read.parquet('../data/raw/green_tripdata_2019-06.parquet').drop("trip_type")
Gjul19 = spark.read.parquet('../data/raw/green_tripdata_2019-07.parquet').drop("trip_type")
Gaug19 = spark.read.parquet('../data/raw/green_tripdata_2019-08.parquet').drop("trip_type")
Gsep19 = spark.read.parquet('../data/raw/green_tripdata_2019-09.parquet').drop("trip_type")
Goct19 = spark.read.parquet('../data/raw/green_tripdata_2019-10.parquet').drop("trip_type")
Gnov19 = spark.read.parquet('../data/raw/green_tripdata_2019-11.parquet').drop("trip_type")
Gdec19 = spark.read.parquet('../data/raw/green_tripdata_2019-12.parquet').drop("trip_type")

Gjan18 = spark.read.parquet('../data/raw/green_tripdata_2018-01.parquet').drop("trip_type")
Gfeb18 = spark.read.parquet('../data/raw/green_tripdata_2018-02.parquet').drop("trip_type")
Gmar18 = spark.read.parquet('../data/raw/green_tripdata_2018-03.parquet').drop("trip_type")
Gapr18 = spark.read.parquet('../data/raw/green_tripdata_2018-04.parquet').drop("trip_type")
Gmay18 = spark.read.parquet('../data/raw/green_tripdata_2018-05.parquet').drop("trip_type")
Gjun18 = spark.read.parquet('../data/raw/green_tripdata_2018-06.parquet').drop("trip_type")
Gjul18 = spark.read.parquet('../data/raw/green_tripdata_2018-07.parquet').drop("trip_type")
Gaug18 = spark.read.parquet('../data/raw/green_tripdata_2018-08.parquet').drop("trip_type")
Gsep18 = spark.read.parquet('../data/raw/green_tripdata_2018-09.parquet').drop("trip_type")
Goct18 = spark.read.parquet('../data/raw/green_tripdata_2018-10.parquet').drop("trip_type")
Gnov18 = spark.read.parquet('../data/raw/green_tripdata_2018-11.parquet').drop("trip_type")
Gdec18 = spark.read.parquet('../data/raw/green_tripdata_2018-12.parquet').drop("trip_type")

Yjan19 = spark.read.parquet('../data/raw/yellow_tripdata_2019-01.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Yfeb19 = spark.read.parquet('../data/raw/yellow_tripdata_2019-02.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Ymar19 = spark.read.parquet('../data/raw/yellow_tripdata_2019-03.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Yapr19 = spark.read.parquet('../data/raw/yellow_tripdata_2019-04.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Ymay19 = spark.read.parquet('../data/raw/yellow_tripdata_2019-05.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Yjun19 = spark.read.parquet('../data/raw/yellow_tripdata_2019-06.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Yjul19 = spark.read.parquet('../data/raw/yellow_tripdata_2019-07.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Yaug19 = spark.read.parquet('../data/raw/yellow_tripdata_2019-08.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Ysep19 = spark.read.parquet('../data/raw/yellow_tripdata_2019-09.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Yoct19 = spark.read.parquet('../data/raw/yellow_tripdata_2019-10.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Ynov19 = spark.read.parquet('../data/raw/yellow_tripdata_2019-11.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Ydec19 = spark.read.parquet('../data/raw/yellow_tripdata_2019-12.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")

Yjan18 = spark.read.parquet('../data/raw/yellow_tripdata_2019-01.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Yfeb18 = spark.read.parquet('../data/raw/yellow_tripdata_2019-02.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Ymar18 = spark.read.parquet('../data/raw/yellow_tripdata_2019-03.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Yapr18 = spark.read.parquet('../data/raw/yellow_tripdata_2019-04.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Ymay18 = spark.read.parquet('../data/raw/yellow_tripdata_2019-05.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Yjun18 = spark.read.parquet('../data/raw/yellow_tripdata_2019-06.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Yjul18 = spark.read.parquet('../data/raw/yellow_tripdata_2019-07.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Yaug18 = spark.read.parquet('../data/raw/yellow_tripdata_2019-08.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Ysep18 = spark.read.parquet('../data/raw/yellow_tripdata_2019-09.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Yoct18 = spark.read.parquet('../data/raw/yellow_tripdata_2019-10.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Ynov18 = spark.read.parquet('../data/raw/yellow_tripdata_2019-11.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")
Ydec18 = spark.read.parquet('../data/raw/yellow_tripdata_2019-12.parquet').withColumn('ehail_fee', lit(0)).drop("airport_fee")

In [4]:
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [5]:
sdf = unionAll(*[ \
     Gjan19, Gfeb19, Gmar19, Gapr19, Gmay19, Gjun19, Gjul19, Gaug19, Gsep19, Goct19, Gnov19, Gdec19,\
     Gjan18, Gfeb18, Gmar18, Gapr18, Gmay18, Gjun18, Gjul18, Gaug18, Gsep18, Goct18, Gnov18, Gdec18, \
     Yjan19, Yfeb19, Ymar19, Yapr19, Ymay19, Yjun19, Yjul19, Yaug19, Ysep19, Yoct19, Ynov19, Ydec19,\
     Yjan18, Yfeb18, Ymar18, Yapr18, Ymay18, Yjun18, Yjul18, Yaug18, Ysep18, Yoct18, Ynov18, Ydec18,])
sdf.count()

184397591

### Dropped Columns:

VendorID is used to indicate the provider who indicated the record, this inforation is not nesscary to know for this project as it has no effect of the taxis themselves. 

RatecodeID is used to indicate the kind of rate used (to calculate payment) during the ride. This is unescary information due to other columns.

store_and_fwd_flag is used to flag if the vechile had a conection to the service during payment, this information is unnessacy.

In [6]:
sdf = sdf.drop("VendorID", "RatecodeID", "store_and_fwd_flag")

### Combined columns

improvement_surcharge, mta_tax, extra, tolls_amount, ehail_fee and congestion_surcharge are all various surcharges applied to the total fee to the customer. These on their own dont hold alot of information but the total will be kept.

In [7]:
from pyspark.sql.functions import *
sdf = sdf.na.fill(value=0,subset=["congestion_surcharge", "ehail_fee"])
sdf = sdf.withColumn('surcharge_amount', col("improvement_surcharge") + col("mta_tax") + col("extra") + col("tolls_amount") + col("ehail_fee") + col("congestion_surcharge"))
sdf = sdf.drop("extra", "mta_tax", "improvement_surcharge", "congestion_surcharge", "tolls_amount", "ehail_fee")

### Renaming records

In [8]:
sdf = sdf.withColumnRenamed("lpep_pickup_datetime", "pickup_time")
sdf = sdf.withColumnRenamed("lpep_dropoff_datetime", "dropoff_time")
sdf = sdf.withColumnRenamed("PULocationID", "pickup_location")
sdf = sdf.withColumnRenamed("DOLocationID", "dropoff_location")

### Converting units

The trip_distance is set to miles (imperial), thus for ease of analysis, this will be set to kilometres (metric)

In [9]:
sdf = sdf.withColumn("trip_distance_km", round(col("trip_distance") * 1.60934, 3)) 
sdf = sdf.drop('trip_distance')

### Adding Trip duration (in minutes)

As there are time stamps of the pickup an dropoff times, it may be handy to look into aspects affecting the duration of the trip, thus we will add a column for total duration in minutes:

In [10]:
from pyspark.sql.functions import *
sdf = sdf.withColumn('trip_time_sec',unix_timestamp("dropoff_time") - unix_timestamp('pickup_time'))
sdf = sdf.withColumn('trip_time_min', round(col('trip_time_sec') / 60, 3))
sdf = sdf.drop('trip_time_sec')

### Adjusting column types

In [11]:
sdf.printSchema()

root
 |-- pickup_time: timestamp (nullable = true)
 |-- dropoff_time: timestamp (nullable = true)
 |-- pickup_location: double (nullable = true)
 |-- dropoff_location: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- surcharge_amount: double (nullable = true)
 |-- trip_distance_km: double (nullable = true)
 |-- trip_time_min: double (nullable = true)



In [12]:
sdf = sdf.withColumn('passenger_count', sdf.passenger_count.cast('int'))
sdf = sdf.withColumn('pickup_location', sdf.pickup_location.cast('long'))
sdf = sdf.withColumn('dropoff_location', sdf.pickup_location.cast('long'))
sdf = sdf.withColumn('payment_type', sdf.payment_type.cast('int'))
sdf = sdf.withColumn('payment_type', sdf.payment_type.cast('string'))

### Invalidating data
#### passenger_count:

As the reasonable minimum nuber of passengers is 1, For taxis in NYC the maximum legal number of passengers is 5, thus we will onyl keep instances between 1 and 5 passengers:

In [13]:
sdf = sdf.filter(sdf.passenger_count > 0)
sdf = sdf.filter(sdf.passenger_count < 6)


#### pickup/dropoff time:
ensuring all data comes from the two year window:

In [14]:
sdf = sdf.filter(sdf.pickup_time >= '2018-01-01 00:00:00')
sdf = sdf.filter(sdf.pickup_time < '2020-01-01 00:00:00')
sdf = sdf.filter(sdf.dropoff_time >= '2018-01-01 00:00:00')
sdf = sdf.filter(sdf.dropoff_time < '2020-01-01 00:00:00')


#### Trip_distance:

As New York City from north to south (Tottenville to Wakefield) is approximately 75km long trip according to https://www.taxi-calculator.com/taxi-fare-estimation, we will set that as the upper bound of acceptable taxi trip distances, and of course all non-negative values as the lower bound:

In [15]:
sdf = sdf.filter(sdf.trip_distance_km > 0)
sdf = sdf.filter(sdf.trip_distance_km < 75)

### Total_amount:

According to https://www.taxifarefinder.com/main.php?city=NY&from=Conference+House+Park%2C+7455+Hylan+Blvd%2C+New+York+City%2C+New+York%2C+10307%2C+United+States+of+America&to=Metro+North+-+Wakefield+Station%2C+E+241st+St%2FWakefield+Ave%2C+New+York+City%2C+New+York%2C+10470%2C+United+States+of+America&fromCoord=40.502859,-74.252418&toCoord=40.905372,-73.85469 the longest trip would cost $129.72 to travel excluding tips, thus assuming a very generous tip of $100, we will set our upper most boundary at $230.

In [16]:
sdf = sdf.filter(sdf.total_amount > 0)
sdf = sdf.filter(sdf.total_amount < 230)

### fare_amount:
Although some taxi rides may be given as free, these are clearly outliers and wont follow tipical trends and thus should be removed.

In [17]:
sdf = sdf.filter(sdf.fare_amount > 0)

#### Trip_time:

Based on the reasonable maximum travel diatance of 75km, we assume an average travel speed of 40km/h (condering extremely bad traffic as well as highways) which works out to be approximately 2 hours, thus we can set this as the reasonable upper bound and 0 as the reasonable lower bound

In [18]:
sdf = sdf.filter(sdf.trip_time_min > 0)
sdf = sdf.filter(sdf.trip_time_min < 120)

#### Payment Type:

According to the data dictionary, only payment types of 1 (credit card) has records of tips thus we will only keep payments type 1 and then we wil drop the column from thw dataset.

In [19]:
sdf = sdf.filter(sdf.payment_type == 1)
sdf = sdf.drop('payment_type')

In [23]:
from pyspark.sql.functions import date_format
import pyspark.sql.functions as F
sdf = sdf.withColumn('pickup_hour_time', date_format('pickup_time', 'HH'))
sdf = sdf.withColumn('pickup_hour_time', F.regexp_replace('pickup_hour_time', r'^(0){1}', ''))
sdf = sdf.withColumn('pickup_hour_time', sdf.pickup_hour_time.cast('int'))

### Outputting DataFrame:

In [26]:
sdf.write.parquet("../data/curated/taxi")

22/08/16 08:48:21 WARN DAGScheduler: Broadcasting large task binary with size 1577.2 KiB


In [20]:
sdf.count()

7882213